In [ ]:
def Download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number):
    from snappy import ProductIO
    from snappy import jpy
    from snappy import GPF

    import numpy as np

    from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
    from datetime import date

    import zipfile
    import os 
    #user = 'jonathanc'
    #password = 'Tutorat2019'
    #api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
    #url = 'https://coda.eumetsat.int'
    api = SentinelAPI(user, password, url)

    #footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
    #date = ('20191001','NOW')
    products = api.query(footprint,
                         date = (datestart,dateend),
                         platformname = 'Sentinel-3',
                         area_relation='Intersects', 
                         producttype='OL_2_WFR___',
                         #timeliness='Non Time Critical',
                         instrumentshortname = 'OLCI',
                         onlinequalitycheck = 'PASSED',
                         productlevel = 'L2')
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(products)
    # sort and limit to first 5 sorted products
    products_df_sorted = products_df.sort_values(['salinewatercoverpercentage', 'ingestiondate'], ascending=[False, True])
    products_df_sorted = products_df_sorted.head(image_number)
    # download sorted and reduced products
    api.download_all(products_df_sorted.index)
    #zip_file_path = "/home/courtois/jupyter-sentinel"
    file_list = os.listdir(zip_file_path+"/")
    # select the directory where the files are and where you want to unzip it
    abs_path = []
    # list for the .zip file to unzip
    c=0
    for a in file_list:
        if a.endswith(".zip"):  # select only the .zip files in the folder
            x = zip_file_path+'/'+a
            c += 1
            print x
            abs_path.append(x)  # put the .zip file in the list
    for f in abs_path:
        zip=zipfile.ZipFile(f)
        zip.extractall(zip_file_path)  # unzip the file in the zip_file_path
        os.remove(f)   # delet the .zip file
        
    print("File in :" + zip_file_path)
    print("DONE")

In [ ]:
user = 'jonathanc'
password = 'Tutorat2019'
url = 'https://coda.eumetsat.int'
#delta du rhone
footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
#australie mundo island
footprint = 'POLYGON ((138.9714316874765 -35.12369943285148, 139.587378163029 -35.28580919523616,139.34742465145243 -35.900220961012316, 138.72728634440375 -35.73680981767152,138.9714316874765 -35.12369943285148))'
datestart = ('20191001')
dateend = ('NOW')
zip_file_path = "/home/courtois/jupyter-sentinel"
image_number = 10
Download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number)